## Базовий arc-eager dependency parser 

In [1]:
from collections import OrderedDict
from unlabeledparser import ArcEagerDependencyParser, update_relations
from metrics import uas, uas_parse, print_tree
from conllu import parse
from pymorphy2tree import Pymorhy2Tree
from uddisplacy import ud_to_displacy
from sklearn.metrics import classification_report
import logging
import pandas as pd
PATH = "data\\UD_Ukrainian-IU-master\\UD_Ukrainian-IU-master"

In [2]:
with open(PATH + "\\uk_iu-ud-train.conllu", "r", encoding='utf-8') as f:
    data = f.read()
train_trees = parse(data)
print(len(train_trees))

4513


In [3]:
with open(PATH + "\\uk_iu-ud-test.conllu", "r", encoding='utf-8') as f:
    data = f.read()
test_trees = parse(data)
print(len(test_trees))

783


In [4]:
dp = ArcEagerDependencyParser()
# dp.train(train_trees)
dp.load('dp_vectorizer_20180515_183226.pkl', 'dp_classifier_20180515_183227.pkl')

In [5]:
predicted, test_labels = dp.test(test_trees)
print(classification_report(test_labels, predicted))

             precision    recall  f1-score   support

       left       0.84      0.86      0.85      7352
     reduce       0.81      0.73      0.77      8370
      right       0.72      0.78      0.75      7182
      shift       0.86      0.86      0.86      7757

avg / total       0.81      0.81      0.81     30661



In [6]:
uas_parse(test_trees, dp.dep_parse, log=True);

Total: 14939
Correctly defined: 10236
UAS: 0.69


### Результат парсингу, використовуючи лише надані pymorphy2 характеристики

In [7]:
pm = Pymorhy2Tree()
test_trees_pm = []
for tree in test_trees:
    test_trees_pm.append(pm.parse_tree_connlu(tree))
print(len(test_trees_pm))

783


In [8]:
predicted, labels = dp.test(test_trees_pm)
print(classification_report(test_labels, predicted))

             precision    recall  f1-score   support

       left       0.36      0.16      0.22      7352
     reduce       0.38      0.84      0.52      8370
      right       0.31      0.09      0.14      7182
      shift       0.42      0.37      0.39      7757

avg / total       0.37      0.38      0.33     30661



In [9]:
good_trees = []
bad_trees = []
for i, tree in enumerate(test_trees_pm):
    try:
        update_relations(tree, dp.dep_parse(tree))
        good_trees.append((tree, test_trees[i]))
    except:
        logging.exception('Can not parse a tree')
        bad_trees.append((tree, test_trees[i]))


ERROR:root:Can not parse a tree
Traceback (most recent call last):
  File "<ipython-input-9-250bd2851f1b>", line 5, in <module>
    update_relations(tree, dp.dep_parse(tree))
  File "c:\work\jul\prj-nlp\students\juliamakogon\task_08\unlabeledparser.py", line 192, in dep_parse
    stack.pop()
IndexError: pop from empty list
ERROR:root:Can not parse a tree
Traceback (most recent call last):
  File "<ipython-input-9-250bd2851f1b>", line 5, in <module>
    update_relations(tree, dp.dep_parse(tree))
  File "c:\work\jul\prj-nlp\students\juliamakogon\task_08\unlabeledparser.py", line 192, in dep_parse
    stack.pop()
IndexError: pop from empty list
ERROR:root:Can not parse a tree
Traceback (most recent call last):
  File "<ipython-input-9-250bd2851f1b>", line 5, in <module>
    update_relations(tree, dp.dep_parse(tree))
  File "c:\work\jul\prj-nlp\students\juliamakogon\task_08\unlabeledparser.py", line 192, in dep_parse
    stack.pop()
IndexError: pop from empty list
ERROR:root:Can not parse 

IndexError: pop from empty list
ERROR:root:Can not parse a tree
Traceback (most recent call last):
  File "<ipython-input-9-250bd2851f1b>", line 5, in <module>
    update_relations(tree, dp.dep_parse(tree))
  File "c:\work\jul\prj-nlp\students\juliamakogon\task_08\unlabeledparser.py", line 192, in dep_parse
    stack.pop()
IndexError: pop from empty list
ERROR:root:Can not parse a tree
Traceback (most recent call last):
  File "<ipython-input-9-250bd2851f1b>", line 5, in <module>
    update_relations(tree, dp.dep_parse(tree))
  File "c:\work\jul\prj-nlp\students\juliamakogon\task_08\unlabeledparser.py", line 192, in dep_parse
    stack.pop()
IndexError: pop from empty list
ERROR:root:Can not parse a tree
Traceback (most recent call last):
  File "<ipython-input-9-250bd2851f1b>", line 5, in <module>
    update_relations(tree, dp.dep_parse(tree))
  File "c:\work\jul\prj-nlp\students\juliamakogon\task_08\unlabeledparser.py", line 192, in dep_parse
    stack.pop()
IndexError: pop from empt

In [10]:
print('Trees parsed without exceptions:', len(good_trees))
print('Trees parsed with exceptions:', len(bad_trees))

Trees parsed without exceptions: 747
Trees parsed with exceptions: 36


In [11]:
print('UAS for all the test trees')
uas(test_trees_pm, test_trees, log=True);

UAS for all the test trees
Total: 14939
Correctly defined: 6314
UAS: 0.42


In [12]:
def uas_good_trees(good_trees):
    trees = [x[0] for x in good_trees]
    golden_trees = [x[1] for x in good_trees]
    uas(trees, golden_trees, log=True);

print('UAS for the test trees parsed without exceptions')
uas_good_trees(good_trees)

UAS for the test trees parsed without exceptions
Total: 13945
Correctly defined: 6314
UAS: 0.45


#### Дерева, з якими парсер впорався без виключень

In [13]:
print('Without exception:')
uas([good_trees[0][0]], [good_trees[0][1]], log= True)
print('\nGold tree')
print_tree(good_trees[0][1])
print('\nParsed tree')
print_tree(good_trees[0][0])

Without exception:
Total: 10
Correctly defined: 8
UAS: 0.8

Gold tree
Зречення <-- втрата
культурної <-- ідентичності
ідентичності <-- Зречення
— <-- втрата
це <-- втрата
втрата <-- root
свободи <-- втрата
й <-- самовладності
самовладності <-- свободи
. <-- втрата

Parsed tree
Зречення <-- root
культурної <-- ідентичності
ідентичності <-- Зречення
— <-- втрата
це <-- втрата
втрата <-- root
свободи <-- втрата
й <-- самовладності
самовладності <-- свободи
. <-- root


In [14]:
uas([good_trees[10][0]], [good_trees[10][1]], log= True)
print('\nGold tree')
print_tree(good_trees[10][1])
print('\nParsed tree')
print_tree(good_trees[10][0])

Total: 34
Correctly defined: 22
UAS: 0.65

Gold tree
М <-- розвінчує
. <-- М
Чернишевського <-- М
, <-- кумира
цього <-- кумира
кумира <-- М
російських <-- шістдесятників
шістдесятників <-- кумира
, <-- кумира
вона <-- розвінчує
дуже <-- гостро
гостро <-- розвінчує
розвінчує <-- root
у <-- статті
статті <-- розвінчує
« <-- Утопия
« <-- Утопия
Утопия <-- статті
» <-- Утопия
в <-- Утопия
беллетристическом <-- Утопия
смысле <-- Утопия
» <-- Утопия
, <-- вважаючи
вважаючи <-- розвінчує
, <-- брехав
що <-- брехав
він <-- брехав
« <-- брехав
зумисне <-- брехав
брехав <-- вважаючи
» <-- брехав
читачам <-- брехав
. <-- розвінчує

Parsed tree
М <-- root
. <-- root
Чернишевського <-- М
, <-- кумира
цього <-- кумира
кумира <-- Чернишевського
російських <-- шістдесятників
шістдесятників <-- кумира
, <-- розвінчує
вона <-- розвінчує
дуже <-- гостро
гостро <-- розвінчує
розвінчує <-- М
у <-- статті
статті <-- розвінчує
« <-- Утопия
« <-- Утопия
Утопия <-- розвінчує
» <-- Утопия
в <-- беллетристическ

In [15]:
# golden tree
ud_to_displacy(good_trees[10][1])

None

In [16]:
# parsed tree
ud_to_displacy(good_trees[10][0], show_deprel=False)

None

In [69]:
# real sentence
s = "Друзі, а комусь вдалось працювати з послідовностями різної довжини в Keras?"
def dep_parse_sentence(s, dp):
    tree = pm.parse_sentence(s)
    update_relations(tree, dp.dep_parse(tree))
    return tree

ud_to_displacy(dep_parse_sentence(s, dp))

None

In [70]:
s = 'На вулицях Володимирській і Михайлівській у столиці заборонили паркуватися до 31 травня.'
ud_to_displacy(dep_parse_sentence(s, dp))

None

#### Дерева, що не вдалося розпарсити

In [19]:
# A tree, that was not parsed because of exception
ud_to_displacy(bad_trees[0][1])

None

In [20]:
ud_to_displacy(bad_trees[0][0])

None

In [21]:
ud_to_displacy(bad_trees[1][1])

None

#### Pymorhy2Tree vs ручний розбір

In [22]:
def pos_to_df(trees, golden_trees):
    lines = []
    columns = ['word form', 'golden lemma', 'lemma', 'lemma is correct', 'golden POS', 'POS', 'POS is correct', 'golden features count', 'features count']
    for tree, golden in zip(trees, golden_trees):
        for i, node in enumerate(golden):
            tree_node = tree[i]
            lines.append([node['form'], node['lemma'], tree_node['lemma'], node['lemma'] == tree_node['lemma'],
                         node['upostag'], tree_node['upostag'], node['upostag'] == tree_node['upostag'],
                         len(node['feats']) if node['feats'] else 0, len(tree_node['feats']) if tree_node['feats'] else 0])
    return pd.DataFrame(lines, columns=columns)

df = pos_to_df(test_trees_pm, test_trees)
display(df)

,word form,golden lemma,lemma,lemma is correct,golden POS,POS,POS is correct,golden features count,features count
0,Зречення,зречення,зречення,True,NOUN,NOUN,True,4,3
1,культурної,культурний,культурний,True,ADJ,ADJ,True,4,2
2,ідентичності,ідентичність,ідентичність,True,NOUN,NOUN,True,4,3
3,—,—,—,True,PUNCT,PUNCT,True,1,0
4,це,це,це,True,PRON,PART,False,5,0
5,втрата,втрата,втрата,True,NOUN,NOUN,True,4,3
6,свободи,свобода,свобода,True,NOUN,NOUN,True,4,3
7,й,й,й,True,CCONJ,CCONJ,True,0,0
8,самовладності,самовладність,самовладність,True,NOUN,NOUN,True,4,3
9,.,.,.,True,PUNCT,PUNCT,True,0,0


In [23]:
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None) 

In [24]:
display(df.groupby(['lemma is correct']).count())
print('Accuracy: ', 13290/14939)

,word form,golden lemma,lemma,golden POS,POS,POS is correct,golden features count,features count
lemma is correct,,,,,,,,
False,1649,1649,1649,1649,1649,1649,1649,1649
True,13290,13290,13290,13290,13290,13290,13290,13290


Accuracy:  0.8896177789678024


In [25]:
display(df.groupby(['POS is correct']).count())
print('Accuracy: ', 12030/14939)

,word form,golden lemma,lemma,lemma is correct,golden POS,POS,golden features count,features count
POS is correct,,,,,,,,
False,2909,2909,2909,2909,2909,2909,2909,2909
True,12030,12030,12030,12030,12030,12030,12030,12030


Accuracy:  0.8052747841220965


In [26]:
display(df[df['POS is correct']==False].groupby(['golden POS', 'POS']).sum())

lemma is correct  POS is correct  golden features count  \
golden POS POS                                                              
ADJ        ADV                 1.0             0.0                     26   
           INTJ                0.0             0.0                      4   
           NOUN                4.0             0.0                     99   
           NUM               102.0             0.0                    530   
           PRON                0.0             0.0                      6   
           VERB                0.0             0.0                      7   
           X                   0.0             0.0                     25   
ADP        ADV               124.0             0.0                    124   
           INTJ              228.0             0.0                    228   
           NOUN              208.0             0.0                    219   
           VERB                0.0             0.0                     13   
           X                   1.0             0.0                      1   
ADV        ADJ                27.0             0.0                     28   
           ADP                 1.0             0.0                      1   
           CCONJ              19.0             0.0                      0   
           INTJ                1.0             0.0                      0   
           NOUN               24.0             0.0                     32   
           PART               89.0             0.0                     60   
           PRON               55.0             0.0                     57   
           SCONJ               5.0             0.0                      4   
           X                   9.0             0.0                     10   
AUX        ADJ                 0.0             0.0                     66   
           NOUN                0.0             0.0                     28   
           PART                7.0             0.0                      7   
           VERB               76.0             0.0                    447   
CCONJ      PART               20.0             0.0                      0   
           SCONJ               2.0             0.0                      0   
           X                   1.0             0.0                      0   
DET        ADJ                 6.0             0.0                     20   
           ADV                 4.0             0.0                     12   
           CCONJ               0.0             0.0                      4   
           NOUN                5.0             0.0                    313   
           PART                0.0             0.0                     30   
           PRON              458.0             0.0                   2125   
INTJ       PART                4.0             0.0                      0   
NOUN       ADJ                17.0             0.0                    310   
           ADV                 1.0             0.0                      8   
           INTJ                1.0             0.0                     27   
           NUM                 2.0             0.0                     12   
           PART                0.0             0.0                     30   
           PUNCT              23.0             0.0                    115   
           VERB                0.0             0.0                    192   
           X                  32.0             0.0                    282   
NUM        NOUN                1.0             0.0                      6   
           PRON                7.0             0.0                     29   
           VERB                0.0             0.0                      8   
           X                   0.0             0.0                      9   
PART       ADV                 2.0             0.0                      0   
           CCONJ              52.0             0.0                      0   
           INTJ                2.0             0.0                      0   
           NOU

In [27]:
display(df[df['POS is correct']==False].groupby(['golden POS', 'POS']).count())

word form  golden lemma  lemma  lemma is correct  \
golden POS POS                                                       
ADJ        ADV           23            23     23                23   
           INTJ           1             1      1                 1   
           NOUN          34            34     34                34   
           NUM          106           106    106               106   
           PRON           2             2      2                 2   
           VERB           2             2      2                 2   
           X              5             5      5                 5   
ADP        ADV          124           124    124               124   
           INTJ         228           228    228               228   
           NOUN         219           219    219               219   
           VERB          13            13     13                13   
           X              1             1      1                 1   
ADV        ADJ           32            32     32                32   
           ADP            1             1      1                 1   
           CCONJ         19            19     19                19   
           INTJ           1             1      1                 1   
           NOUN          68            68     68                68   
           PART          89            89     89                89   
           PRON          57            57     57                57   
           SCONJ          5             5      5                 5   
           X             16            16     16                16   
AUX        ADJ           11            11     11                11   
           NOUN          12            12     12                12   
           PART           7             7      7                 7   
           VERB          77            77     77                77   
CCONJ      PART          20            20     20                20   
           SCONJ          2             2      2                 2   
           X              1             1      1                 1   
DET        ADJ            6             6      6                 6   
           ADV            4             4      4                 4   
           CCONJ          1             1      1                 1   
           NOUN          56            56     56                56   
           PART           7             7      7                 7   
           PRON         478           478    478               478   
INTJ       PART           4             4      4                 4   
NOUN       ADJ           72            72     72                72   
           ADV            2             2      2                 2   
           INTJ           6             6      6                 6   
           NUM            2             2      2                 2   
           PART           5             5      5                 5   
           PUNCT         23            23     23                23   
           VERB          48            48     48                48   
           X             53            53     53                53   
NUM        NOUN           3             3      3                 3   
           PRON          10            10     10                10   
           VERB           4             4      4                 4   
           X              3             3      3                 3   
PART       ADV            2             2      2                 2   
           CCONJ         52            52     52                52   
           INTJ           2             2      2                 2   
           NOUN           1             1      1                 1   
           PRON           3             3      3                 3   
           SCONJ         16            16     16                16   
           VERB           1             1      1                 1   
           X              1             1      1                 1   
PRON       ADV            6 

#### Висновки
Клас Pymorhy2Tree виконує: 1) морфологічний розбір слова з допомогою pymorphy2; 2) переклад тегів OpenCorpora на Universal dependencies (на даному етапі крім тегів p.tag.mood, p.tag.transitivity, p.tag.voice, p.tag.involvement). 
Для дерев золотого стандарту, де морфорогічні характеристики отримані вручну, маємо UAS=0.69. Для розмічених Pymorhy2Tree маємо UAS=0.42. Працюючи над покращенням морфологічного розбору, можемо отримати значно якісніший синтаксичний розбір на реальних текстах. 

5% дерев не вдалося розпарсити з допомогою ознак з Pymorhy2Tree. Сподіваюся згодом знайти час і розібратися, чому. 

## Більше характеристик
Додаємо: 
* історію stack, queue
* n-грами для вершин стеку та черги


In [28]:
from unlabeledparser import extract_features, oracle, get_root, trace_actions, get_data
from sklearn.linear_model import LogisticRegression
from collections import deque

In [29]:
trace_actions(train_trees[0], get_root(), oracle)

Stack: ['ROOT']
Queue: ['У', 'домі', 'римського', 'патриція', 'Руфіна', 'була', 'прегарна', 'фреска', ',', 'зображення', 'Венери', 'та', 'Адоніса', '.']
Relations: []
Actions.SHIFT
Stack: ['ROOT', 'У']
Queue: ['домі', 'римського', 'патриція', 'Руфіна', 'була', 'прегарна', 'фреска', ',', 'зображення', 'Венери', 'та', 'Адоніса', '.']
Relations: []
Actions.LEFT
Stack: ['ROOT']
Queue: ['домі', 'римського', 'патриція', 'Руфіна', 'була', 'прегарна', 'фреска', ',', 'зображення', 'Венери', 'та', 'Адоніса', '.']
Relations: [(1, 2)]
Actions.SHIFT
Stack: ['ROOT', 'домі']
Queue: ['римського', 'патриція', 'Руфіна', 'була', 'прегарна', 'фреска', ',', 'зображення', 'Венери', 'та', 'Адоніса', '.']
Relations: [(1, 2)]
Actions.SHIFT
Stack: ['ROOT', 'домі', 'римського']
Queue: ['патриція', 'Руфіна', 'була', 'прегарна', 'фреска', ',', 'зображення', 'Венери', 'та', 'Адоніса', '.']
Relations: [(1, 2)]
Actions.LEFT
Stack: ['ROOT', 'домі']
Queue: ['патриція', 'Руфіна', 'була', 'прегарна', 'фреска', ',', 'зобр

In [60]:
class MemoryFeatureExtractor():
    def start_sentence(self, sentence):
        self.sentence = sentence
        self.history = deque()
    
    def extract_features(self, stack, queue):
        features = extract_features(stack, queue)
        stack_0 = stack[-1] if stack and len(stack) > 0 else None
        stack_1 = stack[-2] if stack and len(stack) > 1 else None
        queue_0 = queue[0] if queue and len(queue) > 0 else None
        queue_1 = queue[1] if queue and len(queue) > 1 else None
        queue_2 = queue[2] if queue and len(queue) > 2 else None
        #ngrams
        features['q-0-1'] = "{}_{}".format(queue_0['upostag'] if queue_0 else '', queue_1['upostag'] if queue_1 else '')
        features['s-0-1'] = "{}_{}".format(stack_0['upostag'] if stack_0 else '', stack_1['upostag'] if stack_1 else '')
        if stack_0:
            idx = stack_0['id']-1
            s_left = self.sentence[idx - 1] if idx > 0 else None
            s_right = self.sentence[idx + 1] if idx + 1 < len(self.sentence) else None
            features['s-3gram'] = "{}_{}_{}".format(s_left['upostag'] if s_left else '', stack_0['upostag'],
                                                   s_right['upostag'] if s_right else '')
            features['s-2gram-left'] = "{}_{}".format(s_left['upostag'] if s_left else '', stack_0['upostag'])
            features['s-2gram-right'] = "{}_{}".format(stack_0['upostag'], s_right['upostag'] if s_right else '')   
        if queue_0:
            idx = queue_0['id']-1
            q_left = self.sentence[idx - 1] if idx > 0 else None
            q_right = self.sentence[idx + 1] if idx + 1 < len(self.sentence) else None
            features['q-3gram'] = "{}_{}_{}".format(q_left['upostag'] if q_left else '', stack_0['upostag'],
                                                   q_right['upostag'] if q_right else '')
            features['q-2gram-left'] = "{}_{}".format(q_left['upostag'] if q_left else '', stack_0['upostag'])
            features['q-2gram-right'] = "{}_{}".format(stack_0['upostag'], q_right['upostag'] if q_right else '')                                                          
        #history
        for i, x in enumerate(self.history):
            features['history-'+str(i)] = x
        if len(self.history) > 1:
            self.history.popleft()
        self.history.append("{}_{}".format(stack_0['upostag'] if stack_0 else '', queue_0['upostag'] if queue_0 else ''))
        return features
    
get_data(train_trees[:3], oracle, MemoryFeatureExtractor(), get_root)


([{'distance': 1,
   'q-0-1': 'ADP_NOUN',
   'q-2gram-left': '_ROOT',
   'q-2gram-right': 'ROOT_NOUN',
   'q-3gram': '_ROOT_NOUN',
   'q0-Case': 'Loc',
   'q0-lemma': 'у',
   'q0-tag': 'ADP',
   'q0-word': 'У',
   'q1-tag': 'NOUN',
   'q1-word': 'домі',
   'q2-tag': 'ADJ',
   'q3-tag': 'NOUN',
   's-0-1': 'ROOT_',
   's-2gram-left': '_ROOT',
   's-2gram-right': 'ROOT_ADP',
   's-3gram': '_ROOT_ADP',
   's0-lemma': 'ROOT',
   's0-tag': 'ROOT',
   's0-word': 'ROOT'},
  {'distance': 1,
   'history-0': 'ROOT_ADP',
   'q-0-1': 'NOUN_ADJ',
   'q-2gram-left': 'ADP_ADP',
   'q-2gram-right': 'ADP_ADJ',
   'q-3gram': 'ADP_ADP_ADJ',
   'q0-Animacy': 'Inan',
   'q0-Case': 'Loc',
   'q0-Gender': 'Masc',
   'q0-Number': 'Sing',
   'q0-lemma': 'дім',
   'q0-tag': 'NOUN',
   'q0-word': 'домі',
   'q1-tag': 'ADJ',
   'q1-word': 'римського',
   'q2-tag': 'NOUN',
   'q3-tag': 'PROPN',
   's-0-1': 'ADP_ROOT',
   's-2gram-left': '_ADP',
   's-2gram-right': 'ADP_NOUN',
   's-3gram': '_ADP_NOUN',
   's0-Case

In [61]:
dpex = ArcEagerDependencyParser(feature_extractor = MemoryFeatureExtractor())
dpex.train(train_trees)

In [62]:
predicted, test_labels = dpex.test(test_trees)
print(classification_report(test_labels, predicted))

             precision    recall  f1-score   support

       left       0.92      0.93      0.93      7352
     reduce       0.88      0.84      0.86      8370
      right       0.83      0.85      0.84      7182
      shift       0.91      0.91      0.91      7757

avg / total       0.89      0.89      0.89     30661



In [63]:
test_trees_pm = []
for tree in test_trees:
    new_tree = pm.parse_tree_connlu(tree)
    test_trees_pm.append(new_tree)

In [64]:
predicted, labels = dpex.test(test_trees_pm)
print(classification_report(test_labels, predicted))

             precision    recall  f1-score   support

       left       0.35      0.20      0.26      7352
     reduce       0.40      0.82      0.54      8370
      right       0.33      0.11      0.17      7182
      shift       0.42      0.38      0.40      7757

avg / total       0.38      0.39      0.35     30661



In [65]:
uas_parse(test_trees, dpex.dep_parse, log=True); 

Total: 14939
Correctly defined: 11617
UAS: 0.78


In [66]:
uas_parse(test_trees, dpex.dep_parse, log=True, trees=test_trees_pm);

Total: 14939
Correctly defined: 8912
UAS: 0.6


In [71]:
s = "Друзі, а комусь вдалось працювати з послідовностями різної довжини в Keras?"
ud_to_displacy(dep_parse_sentence(s, dpex))

None

In [73]:
s = 'На вулицях Володимирській і Михайлівській у столиці заборонили паркуватися до 31 травня.'
ud_to_displacy(dep_parse_sentence(s, dpex))

None

In [74]:
s = "Були собі дід та баба та дожились уже до того, що й хліба нема."
ud_to_displacy(dep_parse_sentence(s, dpex))

None

In [75]:
s="Біжить та й біжить дорогою, коли це назустріч йому зайчик."
ud_to_displacy(dep_parse_sentence(s, dpex))

None

In [76]:
s="Генпрокурор Юрій Луценко вважає, що середньому класу потрібна стабільна політична сила."
ud_to_displacy(dep_parse_sentence(s, dpex))

None

In [79]:
s="Про це він заявив в інтерв'ю \"Інтерфакс-Україна\"."
ud_to_displacy(dep_parse_sentence(s, dpex))

None

#### Висновки
На тестових реченнях UAS: 0.78.
На тестових реченнях, анотованих pymorphy2 UAS: 0.6

In [80]:
dpex.save('vectorizer_20180516_2349.pql', 'classifier_20180516_2349.pql')